In [1]:
import random
import wx
import requests
from bs4 import BeautifulSoup


class Mainframe(wx.Frame):
    def __init__(self, parent, id, title):
        wx.Frame.__init__(self, parent, id, title, size=(300, 450))
        panel = wx.Panel(self)
        self.count = wx.TextCtrl(panel, id=1, pos=(5, 40))
        self.count.SetHint("시행 횟수 입력")
        self.generate_button = wx.Button(panel, id=2, label='생성', pos=(5, 70))
        self.output = wx.ListBox(panel, id=3, choices=[], pos=(130, 5), size=(250, 400))

        fixed_number_box = wx.StaticBox(panel, id=5, label="고정수", pos=(5, 100), size=(80, 50))
        self.fixed_number_1 = wx.TextCtrl(fixed_number_box, id=6, size=(30, 20), pos=(5, 20))
        self.fixed_number_2 = wx.TextCtrl(fixed_number_box, id=7, size=(30, 20), pos=(40, 20))
        self.fixed_number_check = wx.CheckBox(panel, id=8, pos=(100, 120))

        self.consecutive_number = wx.CheckBox(panel, id=9, pos=(10, 160), label="연속수 제거")

        #self.previous_number = self.extractor()
        self.previous_number = 3,6,9,18,22,35
        previous_number_text = wx.StaticText(panel, id=10, label='이전회차 당첨번호\n' + str(self.previous_number), pos=(5, 5))

        self.duplicates_list = ['0', '1', '2', '3', '4', '5', '6']
        self.duplicates = wx.RadioBox(panel, id=11, label="중복 허용 갯수", pos=(5, 200), size=wx.DefaultSize, choices=self.duplicates_list, majorDimension=3, style=wx.RA_SPECIFY_COLS)

        self.save_button = wx.Button(panel, id=12, label="저장", pos=(5, 310))

        self.Bind(wx.EVT_BUTTON, self.lotto, id=2)
        self.Bind(wx.EVT_BUTTON, self.save, id=12)

    def lotto(self, event):
        total_count = int(self.count.GetValue())
        match_count = 0  # 일치한 번호의 개수
        for i in range(total_count):
            number = set()
            if self.fixed_number_check.GetValue():
                number = self.fixed_number(number)
            while len(number) < 6:
                number.add(random.randint(1, 45))
                if len(number) == 6:
                    intersection = number & set(self.previous_number)
                    if len(intersection) > self.duplicates.GetSelection():
                        number.remove(list(intersection)[0])
                    number = list(number)
                    number.sort()
                    if self.consecutive_number.GetValue():
                        for j in range(len(number) - 1):
                            try:
                                if number[j] + 1 == number[j + 1]:
                                    del number[j + 1]
                            except:
                                pass
                    if not len(number) == 6:
                        number = set(number)
                        if self.fixed_number_check.GetValue():
                            number = self.fixed_number(number)
                        if len(number) == 6:
                            number = list(number)
                            number.sort()
            self.output.Append(str(number))
            if set(self.previous_number) & set(number):  # 이전 당첨 번호와 일치하는 경우
                match_count += 1
            # 일치 확률 계산 및 출력

    def save(self, event):
        with open("lotto_number.txt", "w", encoding="utf-8") as f:
            for i in range(self.output.GetCount()):
                f.write(self.output.GetString(i) + "\n")
        dlg = wx.MessageDialog(self, '저장 완료', '저장', wx.OK | wx.ICON_INFORMATION)
        dlg.ShowModal()
        dlg.Destroy()

    def fixed_number(self, number):
        try:
            fixed_number_1 = int(self.fixed_number_1.GetValue())
            number.add(fixed_number_1)
        except:
            pass
        try:
            fixed_number_2 = int(self.fixed_number_2.GetValue())
            number.add(fixed_number_2)
        except:
            pass
        return number

    def extractor(self):
        url = "https://dhlottery.co.kr/common.do?method=main"
        query = requests.get(url=url)
        soup = BeautifulSoup(query.content, "html.parser")
        no_1 = soup.find("span", {"id": "drwtNo1"})
        no_2 = soup.find("span", {"id": "drwtNo2"})
        no_3 = soup.find("span", {"id": "drwtNo3"})
        no_4 = soup.find("span", {"id": "drwtNo4"})
        no_5 = soup.find("span", {"id": "drwtNo5"})
        no_6 = soup.find("span", {"id": "drwtNo6"})
        extract_number = [int(no_1.get_text()), int(no_2.get_text()), int(no_3.get_text()), int(no_4.get_text()),
                          int(no_5.get_text()), int(no_6.get_text())]
        return extract_number


def run():
    app = wx.App()
    frame = Mainframe(None, -1, 'Lotto Number Generator')
    frame.Show()
    app.MainLoop()


run()